# Вторая часть ноутбука

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.utils import shuffle

In [ ]:
df = pd.read_csv("housingdata.csv")

In [ ]:
X = df.drop('MEDV', axis=1)# independent Variables
y = df.iloc[:, 13] #Dependent or target variables

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [ ]:
seed = 26
X, y = shuffle(X, y, random_state=seed)

Реализуем модели с использованием различных типов ядра (линейного, полиномиального и RBF) и сравним их MAE и RMSE

### Linear kernel

In [ ]:
grd = GridSearchCV(estimator=SVR(kernel='linear'),
                       param_grid={'C': [0.1, 1, 100, 1000],
                                   'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],},
                       cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

In [ ]:
#grd.fit(X,y)
grid_result = grd.fit(X, y)
best_params = grid_result.best_params_
best_svr = SVR(kernel='linear', C=best_params["C"], epsilon=best_params["epsilon"], coef0=0.1, shrinking=True,
                   tol=0.001, cache_size=200, verbose=False, max_iter=-1)

scoring = {
               'abs_error': 'neg_mean_absolute_error',
               'squared_error': 'neg_mean_squared_error'}

scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
scoring = {'abs_error': 'neg_mean_absolute_error',
           'squared_error': 'neg_mean_squared_error'}

scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
LMAE=abs(scores['test_abs_error'].mean())
LRMSE=math.sqrt(abs(scores['test_squared_error'].mean()))

print('MAE of Linear Kernel = ', LMAE)
print('RMSE of Linear kernel = ', LRMSE)
#return "MAE :", abs(scores['test_abs_error'].mean()), "| RMSE :", math.sqrt(abs(scores['test_squared_error'].mean()))
# Run 

MAE of Linear Kernel =  3.4443026114013997
RMSE of Linear kernel =  4.849065624928485


### Polynomial Kernel

In [ ]:
gsc = GridSearchCV(
    estimator=SVR(kernel='poly'),
    param_grid={
        'C': [0.1, 1, 100, 1000],
        'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
        'degree': [2, 3, 4],
        'coef0': [0.1, 0.01, 0.001, 0.0001]},
    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

In [ ]:
grid_result = gsc.fit(X, y)
best_params = grid_result.best_params_
best_svr = SVR(kernel='poly', C=best_params["C"], epsilon=best_params["epsilon"], coef0=best_params["coef0"],
               degree=best_params["degree"], shrinking=True,
               tol=0.001, cache_size=200, verbose=False, max_iter=-1)

scoring = {
    'abs_error': 'neg_mean_absolute_error',
    'squared_error': 'neg_mean_squared_error'}

scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
PMAE=abs(scores['test_abs_error'].mean())
PRMSE=math.sqrt(abs(scores['test_squared_error'].mean()))    
print('MAE of Polynomial Kernel = ', PMAE)
print('RMSE of Polinomial Kernel = ', PRMSE)  

MAE of Polynomial Kernel =  2.106990689972121
RMSE of Polinomial Kernel =  3.2128807618748065


### RBF Kernel

In [ ]:
gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid={
            'C': [0.1, 1, 100, 1000],
            'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
            'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

In [ ]:
grid_result = gsc.fit(X, y)
best_params = grid_result.best_params_
best_svr = SVR(kernel='rbf', C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"],
                   coef0=0.1, shrinking=True,
                   tol=0.001, cache_size=200, verbose=False, max_iter=-1)
scoring = {'abs_error': 'neg_mean_absolute_error',
           'squared_error': 'neg_mean_squared_error'}
scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
MAE=abs(scores['test_abs_error'].mean())
RMSE=math.sqrt(abs(scores['test_squared_error'].mean()))
print('MAE of RBF Kernel = ', MAE)
print('RMSE of RBF Kernel = ', RMSE)
#return "MAE :", abs(scores['test_abs_error'].mean()), "| RMSE :", math.sqrt(abs(scores['test_squared_error'].mean()))

MAE of RBF Kernel =  2.099819260764446
RMSE of RBF Kernel =  3.150020608835843


# Выводы и обсуждения

RBF показал лучший результат

Были проведены эксперименты с методами:
- метод опорных векторов с линейным ядром
- метод опорных векторов с полиномиальным ядром
- метод опорных векторов с ядром rbf 

### С какими проблемами столкнулись?

- Была проблема с визуализацией данных через seaborn.jointplot: во-первых, требуется версия 0.11, которая не поддерживается jupyter-notebook'ом, во-вторых, эта визуализация данных стоит очень дорого по оперативной памяти (наступит момент, когда ядро будет занимать больше 2Гб) -- (ноутбук с анализом выборки - первый отправленый ноутбук)

- Довольно долго собираются данные для модели. Ноутбук с SVR работает примерно 15 минут. примерно 15 минут.